In [56]:
import json
import pandas as pd
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
import missingno as msno
import numpy as np
from math import log2
from sklearn.metrics import mean_absolute_error
import random
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from scipy.spatial.distance import correlation
from surprise import Dataset
from surprise import KNNWithMeans, KNNBasic, KNNWithZScore, KNNBaseline
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from surprise.model_selection import cross_validate
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
import pickle
from surprise.model_selection import train_test_split

In [9]:
pip install --upgrade numpy


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [10]:
df = pd.read_json('Grocery_and_Gourmet_Food_5.json',lines=True)

In [11]:
df.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,5,True,"11 19, 2014",A1QVBUH9E1V6I8,4639725183,Jamshed Mathur,No adverse comment.,Five Stars,1416355200,NaN,NaN,NaN
1,5,True,"10 13, 2016",A3GEOILWLK86XM,4639725183,itsjustme,Gift for college student.,Great product.,1476316800,NaN,NaN,NaN
2,5,True,"11 21, 2015",A32RD6L701BIGP,4639725183,Krystal Clifton,"If you like strong tea, this is for you. It mi...",Strong,1448064000,NaN,NaN,NaN
3,5,True,"08 12, 2015",A2UY1O1FBGKIE6,4639725183,U. Kane,Love the tea. The flavor is way better than th...,Great tea,1439337600,NaN,NaN,NaN
4,5,True,"05 28, 2015",A3QHVBQYDV7Z6U,4639725183,The Nana,I have searched everywhere until I browsed Ama...,This is the tea I remembered!,1432771200,NaN,NaN,NaN


In [12]:
drop_list = ['image','vote','style','reviewText','summary','reviewTime','reviewerName','unixReviewTime']
df.drop(df[drop_list], axis=1, inplace=True)
df.head(5)

,overall,verified,reviewerID,asin
0,5,True,A1QVBUH9E1V6I8,4639725183
1,5,True,A3GEOILWLK86XM,4639725183
2,5,True,A32RD6L701BIGP,4639725183
3,5,True,A2UY1O1FBGKIE6,4639725183
4,5,True,A3QHVBQYDV7Z6U,4639725183


In [13]:
indexVerify = df[df['verified'] == False].index
df.drop(indexVerify , inplace=True)
df

,overall,verified,reviewerID,asin
0,5,True,A1QVBUH9E1V6I8,4639725183
1,5,True,A3GEOILWLK86XM,4639725183
2,5,True,A32RD6L701BIGP,4639725183
3,5,True,A2UY1O1FBGKIE6,4639725183
4,5,True,A3QHVBQYDV7Z6U,4639725183
...,...,...,...,...
1143855,5,True,A223YRQH2Z5T1D,B01HJF6FRA
1143856,5,True,A38GDA4TB9EILT,B01HJF6FRA
1143857,5,True,A2025PN7HDC5BO,B01HJF6FRA
1143858,5,True,A1NY7XWC7EPQOA,B01HJF6FRA


In [ ]:
sparse_matrix = pd.pivot_table(df, values='overall', index='reviewerID', columns='asin')

In [ ]:
sparr = df.apply(pd.arrays.SparseArray)

In [8]:
print (sparr.sparse.density)

0.7943700860827958


In [15]:
df_Top100 = df.groupby('reviewerID').agg(mean_rating = ('overall','mean'), number_of_ratings = ('overall', 'count')).reset_index()
df_Top100 = df_Top100[df_Top100['number_of_ratings']>20]
df_Top100



,reviewerID,mean_rating,number_of_ratings
23,A10030KC6GYK89,4.090909,22
33,A100DXY4SLAMPM,4.954545,22
41,A100RH4M1W1DF0,4.806452,31
42,A100UD67AHFODS,4.720000,25
46,A100WO06OQR8BQ,4.443182,88
...,...,...,...
125106,AZU8GQQW6HESP,4.818182,44
125130,AZV26LP92E6WU,4.865546,119
125132,AZV2U6GU5QA6C,4.720000,25
125165,AZWCQL30IJZR8,4.730769,26


In [16]:
df_final = pd.merge(df, df_Top100[['reviewerID']], on='reviewerID',how='inner')
df_final

,overall,verified,reviewerID,asin
0,5,True,A32RD6L701BIGP,4639725183
1,5,True,A32RD6L701BIGP,B0001LO3FG
2,5,True,A32RD6L701BIGP,B000CRIUNU
3,5,True,A32RD6L701BIGP,B000ES5GMK
4,5,True,A32RD6L701BIGP,B000F4DKAI
...,...,...,...,...
153088,5,True,A1GPKGP5PAYOYM,B00W0C9F5M
153089,5,True,A1GPKGP5PAYOYM,B013TY4VYY
153090,5,True,A1GPKGP5PAYOYM,B01EIHCWLK
153091,5,True,A1GPKGP5PAYOYM,B01EIHCWPQ


In [17]:
print('The ratings dataset has', df_final['reviewerID'].nunique(), 'reviewers')
print('The ratings dataset has', df_final['asin'].nunique(), 'unique products')
print('The unique ratings are', sorted(df_final['overall'].unique()), 'ratings')


The ratings dataset has 4640 reviewers
The ratings dataset has 30884 unique products
The unique ratings are [1, 2, 3, 4, 5] ratings


In [18]:
reviewer_products_ratings = df_final.groupby('reviewerID')[['asin', 'overall']].agg(list)


In [19]:
reviewer_products_ratings = df_final.groupby('reviewerID')
reviewer_products_ratings

In [20]:
le = LabelEncoder()
df_final['reviewerID'] = le.fit_transform(df_final['reviewerID'])
df_final['asin'] = le.fit_transform(df_final['asin'])

In [21]:
def split_train_test(group):
    train = group.sample(frac=0.9, random_state=123)
    test = group.drop(train.index)
    return train, test

In [22]:
train_test_splits = reviewer_products_ratings.apply(split_train_test)


In [23]:
train_test_splits = train_test_splits.reset_index(level=[0], drop=True)


In [24]:
train_df = pd.concat([split[0] for split in train_test_splits])
test_df = pd.concat([split[1] for split in train_test_splits])


In [25]:
test_df

,overall,verified,reviewerID,asin
100852,4,True,0,4894
100863,4,True,0,16423
31083,5,True,1,385
31094,5,True,1,22267
151756,5,True,2,8878
...,...,...,...,...
43388,5,True,4638,18227
43399,5,True,4638,29160
126326,5,True,4639,2602
126337,5,True,4639,10354


In [26]:
ratings_matrix = pd.pivot_table(train_df, values='overall', index='reviewerID', columns='asin')


In [27]:
ratings_matrix

asin,0,1,2,3,4,5,6,7,8,9,...,30873,30874,30875,30876,30877,30878,30879,30880,30882,30883
reviewerID,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4635,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4637,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
user_similarity = ratings_matrix.T.corr()


In [47]:
type(user_similarity)
user_similarity_numpy = user_similarity.to_numpy()

In [54]:
user_similarity_numpy

array([[ 1., nan, nan, ..., nan, nan, nan],
       [nan,  1., nan, ..., nan, nan, nan],
       [nan, nan,  1., ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ...,  1., nan, nan],
       [nan, nan, nan, ..., nan,  1., nan],
       [nan, nan, nan, ..., nan, nan,  1.]])

In [28]:
def similarity_sample(user1, user2):
    common_products = ratings_matrix.loc[user1].dropna().index.intersection(ratings_matrix.loc[user2].dropna().index)
    
    if len(common_products) == 0:
        return 0
    
    user1_ratings = ratings_matrix.loc[user1, common_products]
    user2_ratings = ratings_matrix.loc[user2, common_products]
    return correlation(user1_ratings, user2_ratings)

In [29]:
def predict_rating(user, product):
    similarities = [(other_user, similarity_sample(user, other_user)) for other_user in ratings_matrix.index if other_user != user]
    similarities.sort(key=lambda x: x[1], reverse=True)
    k_similarities = similarities[:10]
    
    if len(k_similarities) == 0:
        return 3.0
    
    numerator = 0.0
    denominator = 0.0
    for other_user, similarity in k_similarities:
        if product in ratings_matrix.loc[other_user].index:
            rating = ratings_matrix.loc[other_user, product]
            if not np.isnan(rating):
                numerator += similarity * rating
                denominator += similarity
    if denominator == 0:
        return 3.0
    else:
        return numerator / denominator


In [31]:
reader = Reader(rating_scale=(1, 5))


In [32]:
data_train = Dataset.load_from_df(df_final[['reviewerID', 'asin', 'overall']], reader)

In [33]:
trainset, testset = train_test_split(data_train, test_size=0.2)

In [52]:
sim_options = {'name': 'cosine', 'user_based': True}
algo = KNNWithMeans(sim_options=sim_options)

In [37]:
sim_options = {'name': 'cosine', 'user_based': True}
algo = KNNBasic(sim_options=sim_options)

In [48]:
sim_options = {'name': 'cosine', 'user_based': True}
algo = KNNBaseline(sim_options=sim_options)

In [53]:
algo.fit(trainset)


Computing the cosine similarity matrix...
Done computing similarity matrix.


In [54]:
predictions = algo.test(testset)


In [40]:
threshold = 4
predicted_labels = [1 if prediction.est >= threshold else 0 for prediction in predictions]
true_labels = [int(rating >= threshold) for (_, _, rating) in testset]

In [55]:
mae = accuracy.mae(predictions)
rmse = accuracy.rmse(predictions)


MAE:  0.6298
RMSE: 0.9670


In [19]:
picked_userid = 1
matrix = df_final.pivot_table(index='reviewerID', columns='asin', values='overall')
matrix

asin,0,1,2,3,4,5,6,7,8,9,...,30874,30875,30876,30877,30878,30879,30880,30881,30882,30883
reviewerID,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4635,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4637,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
df_pickle = pd.read_pickle('meta_Grocery_and_Gourmet_Food.json.to_pickle')

AttributeError: Can't get attribute '_unpickle_block' on <module 'pandas._libs.internals' from '/koko/system/anaconda/envs/python39/lib/python3.9/site-packages/pandas/_libs/internals.cpython-39-x86_64-linux-gnu.so'>

In [65]:
pip install pandas==1.2.5


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 9.7 MB 10.3 MB/s eta 0:00:01
  Attempting uninstall: pandas
    Found existing installation: pandas 1.2.3
    Uninstalling pandas-1.2.3:
      Successfully uninstalled pandas-1.2.3
Note: you may need to restart the kernel to use updated packages.
